In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.types.syft_object import SyftObject

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=f6a390f2286e4a3aa7b6aa92ffd42a47 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=c7bdcb0a85f04ddcab96bb27bfa98e92 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# create datasets

In [4]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.52it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [5]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.73it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/fdf8467cb4bb41f299d4dee22409b176.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=d552f61bab2141a087f10e4af70214a1 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_2639: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_2639
597d86e859174962bebc2650fa6ca0bf
override True
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [7]:
client_low.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [9]:
# syft absolute
from syft.client.syncing import compare_states

low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state = compare_states(low_state, high_state)

In [13]:
# syft absolute
# state.objs_to_sync
from syft.client.syncing import resolve

low_items_to_sync, high_items_to_sync = resolve(diff_state)

```python
class UserCode
    id: UID = f25fa6f3f570442db698cc528af3deb7
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

```

Do you approve moving this object from the HIGH side to the LOW side (approve/deny): 


 approve


```python
class Request:
  id: str = 14937af950174dba9fe759a321e72d44
  request_time: str = 2024-02-20 14:28:42
  updated_at: str = None
  status: str = RequestStatus.PENDING
  changes: str = ['Request to change <b>compute_mean</b> (Pool Id: <b>default-pool</b>) to permission <b>RequestStatus.APPROVED</b>. No nested requests']
  requesting_user_verify_key: str = 1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367

```

Do you approve moving this object from the HIGH side to the LOW side (approve/deny): 


 approve


- What do we do with properties here?
  - just set them to None

In [14]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [15]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location = node_uid
        x.syft_client_verify_key = verify_key
        if hasattr(x, "node_uid"):
            x.node_uid = node_uid

    # TODO: hadnle nested collections

In [16]:
def create_actionobject(action_object, client):
    action_object = action_object.refresh_object()
    res = client.api.services.action.set(action_object)
    print("Created", res)

In [17]:
def sync_items(items, node, client):
    for item in items:
        if isinstance(item, ActionObject):
            create_actionobject(item, client)

        else:
            stash = get_stash_for_item(item, node)

            set_obj_ids(item, client.id, client.verify_key)

            res = stash.delete_by_uid(node.python_node.verify_key, item.id)

            res = stash.set(node.python_node.verify_key, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)

In [18]:
low_items_to_sync

[]

In [19]:
sync_items(low_items_to_sync, node_low, client_low)

sync_items(high_items_to_sync, node_high, client_high)

created syft.service.code.user_code.UserCode
created syft.service.request.request.Request


In [20]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state = compare_states(low_state, high_state)

diff_state.diffs

[syft.service.sync.diff_state.Diff, syft.service.sync.diff_state.Diff]

# Run code high and sync back result

In [21]:
client_high._fetch_api(client_high.credentials)

In [22]:
data_high = client_high.datasets[0].assets[0]

In [23]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = bdba4d7178c442c9b873dd12ce8f5ee9
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [24]:
# wait for the result
job_high.wait()

f25fa6f3f570442db698cc528af3deb7
override True


```python
Pointer
```
17.0

In [25]:
job_high.result.get()

17.0

In [26]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state_2 = compare_states(low_state, high_state)

diff_state_2.diffs

[syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff]

In [27]:
diff_state_2.hierarchies

[[(syft.service.sync.diff_state.Diff, 0)],
 [(syft.service.sync.diff_state.Diff, 0),
  (syft.service.sync.diff_state.Diff, 1),
  (syft.service.sync.diff_state.Diff, 2),
  (syft.service.sync.diff_state.Diff, 3)]]

In [45]:
!pip install markdown

  Using cached Markdown-3.5.2-py3-none-any.whl (103 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [76]:
# third party
from IPython.display import display

md = client_low.code[0]._repr_markdown_()

<p>```python
class UserCode
    id: UID = f25fa6f3f570442db698cc528af3deb7
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']</p>
<pre><code>code:
</code></pre>
<p>@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -&gt; float:
    return data.mean()</p>
<p>```</p>


In [74]:
# third party
from IPython.display import HTML
from IPython.display import display
from pygments import highlight
from pygments.formatters import HtmlFormatter
from pygments.lexers import PythonLexer

code_block = """
class UserCode:
    id: UID = 'adc2b43aded147879c586a1901885ba9'
    service_func_name: str = 'compute_mean'
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()
"""

highlighted_code = highlight(code_block, PythonLexer(), HtmlFormatter(style="friendly"))
styles = HtmlFormatter().get_style_defs(".highlight")
status = "NEW"

content = f"""
<div style='display: flex;'>
  <div style='flex: 15%; padding: 10px;'>
    <p><strong>Status:</strong> {status}</p>
  </div>
  <div style='flex: 85%; padding: 5px; background-color: #f0f0f0;'>
    <style>{styles}</style>
    {highlighted_code}
  </div>
</div>
"""

display(HTML(content))

In [29]:
from syft.service.sync.diff_state import DiffState

from IPython.display import display, Markdown

def resolve_hierarchical(diff_state: DiffState, force_approve: bool = False):
    low_new_objs = []
    high_new_objs = []

    for hierarchy in diff_state.hierarchies:


    for new_obj in state.diffs:
        if new_obj.merge_state == "NEW":
            if new_obj.low_obj is None:
                state_list = low_new_objs
                source = "LOW"
                destination = "HIGH"
                obj_to_sync = new_obj.high_obj
            if new_obj.high_obj is None:
                state_list = high_new_objs
                source = "HIGH"
                destination = "LOW"
                obj_to_sync = new_obj.low_obj
            if hasattr(obj_to_sync, "_repr_markdown_"):
                display(Markdown(obj_to_sync._repr_markdown_()))
            else:
                display(obj_to_sync)

            if force_approve:
                state_list.append(obj_to_sync)
            else:
                print(
                    f"Do you approve moving this object from the {source} side to the {destination} side (approve/deny): ",
                    flush=True,
                )
                while True:
                    decision = input()
                    if decision == "approve":
                        state_list.append(obj_to_sync)
                        break
                    elif decision == "deny":
                        break
                    else:
                        print("Please write `approve` or `deny`:", flush=True)
        if new_obj.merge_state == "DIFF":
            pass

    return low_new_objs, high_new_objs

In [27]:
# items_to_sync = state_2.objs_to_sync()
# items_to_sync.append(job_high.result)
low_items_to_sync, high_items_to_sync = resolve(diff_state_2)

```python
Pointer
```
17.0

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


 approve


```python
class Job:
    id: UID = 2e9c871ae6234e17b1a4f66fca2f889b
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


 approve


```python
class SyftLog:
  id: str = e4164c9abbf64a398b9ff230f41967f5

```

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


 approve


In [28]:
sync_items(low_items_to_sync, node_low, client_low)

Created 17.0
created syft.service.job.job_stash.Job
created syft.service.log.log.SyftLog


In [29]:
# low_node_state = client_low.get_sync_state() -> just a table
# high_node_state = client_high.get_sync_state()
# (low_new_state, high_new_state) = resolve_state(low_node_state, high_node_state)

In [31]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

diff_state_3 = compare_states(low_state, high_state)

diff_state_3.diffs

[syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff,
 syft.service.sync.diff_state.Diff]

# Run code low

In [32]:
private_high.mean()

17.0

In [33]:
job_low = client_low.code.compute_mean(data=data_low, blocking=False)

adc2b43aded147879c586a1901885ba9


SyftWarning: There are existing jobs for this user code, returning the latest one

In [34]:
job_low

```python
class Job:
    id: UID = 2e9c871ae6234e17b1a4f66fca2f889b
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

In [35]:
job_low.result

```python
Pointer
```
17.0

# TODO

- Displaying tables (diff table + state table)
    - Improving table flexibility 
- We're not syncing correct IDs with syncing ActionObjects
- We're not syncing outputpolicy
- Incorrect datetimes between updated_at on requests
- When we create multiple changes in requests, we have to sync those as well
- Object hierarchy is not as clean as we want (both Jobs and Requests are at the top of the hierarchy)
- TODO Merge with Koen's permissions